<a href="https://colab.research.google.com/github/NiklasElsaesser/FaceBias/blob/main/FaceBias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project "Bias in Face-Detection"


## 1. Installing necessary libraries
1.   OpenCV to preprocess the data (pictures)  
2.   numpy to organize the data and link it with the labels



In [ ]:
!pip install opencv-contrib-python
!pip install numpy as np
!pip install tensorflow
!pip install wandb

## 2. Import the W&B library
Initialize the W&B project and connecting it with the project on [W&B](https://wandb.ai/elsaesserniklas/4facesbias/reports/Face-Recognition-Bias-Report---Vmlldzo1NjUzNDg3/edit).

In [ ]:
import wandb
wandb.init(project="4facesbias")

## 3. Mounting the Google Drive
The Pictures get uploaded for the labeling and preproccesing.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 4. Labeling & Preprocessing
The pictures get loaded, preprocessed and labeled according to the folder names.

In [4]:
import os
import cv2
import numpy as np

data = []
labels = []

genders = ["men","women"]
emotions = ["happy","neutral"]

for gender in genders:
    for emotion in emotions:
        folder_path = f'/content/drive/MyDrive/Faces/Dataset/{gender}/{emotion}/'
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
            if img is None:
              print("error")
              continue
            img = cv2.resize(img, (48, 48))
            data.append(img)
            # Create labels by combining gender and emotion indices
            label = genders.index(gender) * len(emotions) + emotions.index(emotion)
            labels.append(label)

data = np.array(data)
data = data.reshape((data.shape[0], 48, 48, 1))
labels = np.array(labels)


error


## 5. Preparing the Data
Now Splitting the Data into a training and test set in an 80 - 20 config.

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

## 6. Training
Building and training of the model.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.callbacks import ModelCheckpoint

model = Sequential() # initializing a sequential model
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1))) #adding a 2D convolutional layer, the input shape is 48X48 and in grayscale, just as preprocessed
model.add(MaxPooling2D(pool_size=(2, 2))) # adding a 2D max-pooling layer with a size of 2X2 to reduce the previous laysers output by taking the max value
model.add(Conv2D(128, (3, 3), activation='relu')) # adding another 2D convolutional layer
model.add(MaxPooling2D(pool_size=(2, 2))) # adding another 2D max-pooling layer
model.add(Flatten()) # flattening the 2D vector into 1D to prepare it for the dense layer
model.add(Dense(128, activation='relu')) # adding a dense (the fully connected) layer with 128 units
model.add(Dense(len(genders) * len(emotions), activation='softmax'))  # output layer with the appropriate number of units for the amount of labels

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # configuring the model for training

wandb_callback = wandb.keras.WandbCallback() # callback to monitor the data in WandB

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

## 7. Run Logging

In [7]:
wandb_callback = wandb.keras.WandbCallback()
#history = model.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(val_images, val_labels), callbacks=[wandb_callback])


## 8. Saving
Saving the trained model to the GDrive, for an easier handling also as an .h5 file.

In [13]:
from google.colab import drive
drive.mount('/content/drive')

# Save the model to Google Drive
model.save('bias_model.keras')
model.save('/content/drive/MyDrive/Faces/Dataset/emotion_model.h5')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
